<a href="https://colab.research.google.com/github/Theonimfi/Text-mining/blob/main/Testing_Text_Mining_Check_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !tar -xvf  '/content/drive/MyDrive/Data Science and AI/Text mining/enwiki20220701-stripped.tgz' -C '/content/drive/MyDrive/Data Science and AI/Text mining/'

In [ ]:
# ! pip install pynvml
# ! git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# ! python rapidsai-csp-utils/colab/env-check.py
# ! bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)
# import condacolab
# condacolab.install()
# import condacolab
# condacolab.check()
#!python rapidsai-csp-utils/colab/install_rapids.py stable

In [ ]:
import json

# Get the data from My drive
file1 = open('/content/drive/MyDrive/Data Science and AI/Text mining_Shared/enwiki20220701-stripped/AB/wiki_63', 'r')
Lines = file1.readlines()
  
content = []
count = 0
# Strips the newline character
for line in Lines:
    content.append(json.loads(line.split('\n')[0]))

In [ ]:
import pandas as pd 

# Add data to a pandas dataframe
df = pd.DataFrame(content)

In [ ]:
(df['text'].values == '').sum() 

98939

Play with the data

In [ ]:
# Import libraries and download example data
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('of')

#defining the function to remove punctuation except dot (.)
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation or i == '.'])
    return punctuationfree

def tokenization(text):
    tokens = re.split('W+',text)
    return tokens

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [ ]:
# Keep only rows with non empty text
df_new = df[df.text != '']
df_new['text_preprocessed'] = df_new['text']

# Reindexing
df_new.index = range(len(df_new))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Preprocessing of text
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: remove_punctuation(x))
nltk.download('punkt')
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: word_tokenize(str(x)))
df_new['text_preprocessed']= df_new['text_preprocessed'].apply(lambda x: remove_stopwords(x))
df_new['text_preprocessed'] = [' '.join(map(str, l)) for l in df_new['text_preprocessed']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
df_new.index = range(len(df_new))
df_new.text_preprocessed[0]

'Clepardia Kraków KS Clepardia Kraków Polish football club based Prądnik Biały district Kraków . They currently play IV Liga fifth tier Polish football league . History . The Krowodrza – Modrzejówka Sports Society founded 1909 one earliest clubs Krakow . Initially ground located Lubelska Street next military hospital later premises current military unit Wrocławska Street . The Prądnicki Sports Club established 1930 KS Prądnik . Its operation interrupted World War II reactivated 1955 . Since 1956 clubs football ground Prądnicka Street . In 1967 merger two clubs InterFactory District Sports Club Clepardia established . Over years club sport sections football handball judo . Currently football sections age levels judo functioning . Squad . Trener'

In [ ]:
import spacy
from spacy import displacy
from itertools import *

# Join text per 2000 document
texts = []
for i in range(0, len(df_new),2000):
	texts.append(" ".join(text for text in df_new.text_preprocessed[i:i+2000]))
 
spacy.require_gpu()
total_entities = []
total_sentences = []
counter = 0
keep_sentences = []
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 100000000
for textt in texts:
  counter +=1
  print("We are in ", counter, "out of ", len(texts))
  doc = nlp(textt)

  # Extract all entity labels from a given doc and add them in key value pairs ie "PERSON": [Barack Obama, Michelle Obama]
  doc_entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}
  # Keep also the sentences of the doc in the dict
  doc_entities["sentences"] = doc.sents
  total_entities.append(doc_entities)

# Uncommenting the below takes a lot of time but we have the sentences we need

  # for sent in doc.sents:
  #   count_people = 0
  #   sentence_entities = []
  #   entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(sent.ents, key=lambda x: x.label_), lambda x: x.label_)}
  #   # total_entities.append(entities)
  #   # total_sentences.append(sent)
  #   if 'PERSON' in entities:
  #     if len(entities['PERSON']) > 1:
  #       keep_sentences.append(sent)


We are in  1 out of  26
We are in  2 out of  26
We are in  3 out of  26
We are in  4 out of  26
We are in  5 out of  26
We are in  6 out of  26
We are in  7 out of  26
We are in  8 out of  26
We are in  9 out of  26
We are in  10 out of  26
We are in  11 out of  26
We are in  12 out of  26
We are in  13 out of  26
We are in  14 out of  26
We are in  15 out of  26
We are in  16 out of  26
We are in  17 out of  26
We are in  18 out of  26
We are in  19 out of  26
We are in  20 out of  26
We are in  21 out of  26
We are in  22 out of  26
We are in  23 out of  26
We are in  24 out of  26
We are in  25 out of  26
We are in  26 out of  26


In [ ]:
# Takes a lot of time to get the sentences that we need
keep_sentences = []
total_count = 0
for entity in total_entities:
  total_count += 1
  print("We are in ",total_count, "out of", len(total_entities)) 
  for sentence in entity['sentences']:
    people_in_sentence = sum(person in str(sentence) for person in entity['PERSON'])
    if people_in_sentence > 1:
      keep_sentences.append(str(sentence))

We are in  1 out of 26
We are in  2 out of 26
We are in  3 out of 26
We are in  4 out of 26


In [ ]:
test = str(keep_sentences[34])
# test = remove_punctuation(test)
# test = word_tokenize(str(test))
# test = remove_stopwords(test)
# test = ' '.join(test)

In [ ]:
displacy.render(nlp(str(test)), jupyter=True, style='ent')

In [ ]:
displacy.render(nlp(str(keep_sentences[48])), jupyter=True, style='ent')

In [ ]:
# caption_doc = nlp(textt)

# print('=== Parts of speech ===')
# for token in caption_doc:
#     print(token.text, token.pos_)

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large',device=0)

In [ ]:
print(keep_sentences[152])

In [ ]:
sentences = keep_sentences
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
for sentence in sentences:  

  sentence = str(sentence)
  # We need to use the tokenizer manually since we need special tokens.
  extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(sentence,
                                                                              return_tensors=True, return_text=False)[0]["generated_token_ids"]])
  extracted_triplets = extract_triplets(extracted_text[0])
  print(extracted_triplets)